In [1]:
# Necessary import
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [2]:
# find paths of the datases
person_path = os.path.abspath(os.path.join(os.getcwd(), "../..")) + '/sport_datasets/' + 'person.csv'
membership_path = os.path.abspath(os.path.join(os.getcwd(), "../..")) + '/sport_datasets/' + 'membership.csv'
stat_path = os.path.abspath(os.path.join(os.getcwd(), "../..")) + '/sport_datasets/' + 'stat.csv'

In [3]:
# load data from datasets
person = pd.read_csv(person_path)
membership = pd.read_csv(membership_path)
stat = pd.read_csv(stat_path)

In [4]:
person.head()

,id,firstName,lastName,matchName,type,position,lastUpdated,nationality,nationalityId,dateOfBirth,placeOfBirth,countryOfBirthId,countryOfBirth,height,weight,gender,status,foot
0,dkh5rfgsoiovvuackx9o5svrp,Dawid,Smug,D. Smug,player,Goalkeeper,2018-11-17T18:58:14Z,Poland,27baqb7359ywdvsxxklx1kn11,1994-01-14,Konin,27baqb7359ywdvsxxklx1kn11,Poland,194.0,85.0,Male,active,NaN
1,2m1fi26tv277k17l9v89uux1x,Sadiq,Quliyev,S. Quliyev,player,Defender,2018-12-09T12:31:37Z,Azerbaijan,52sufqqq7vid3xa6iljx6obcz,1995-03-09,NaN,52sufqqq7vid3xa6iljx6obcz,Azerbaijan,NaN,NaN,Male,active,NaN
2,5ojtb45gyd00i0chyku6btuxh,Jaime José,Moreno Ciorciari,J. Moreno,player,Attacker,2018-12-10T05:46:18Z,Nicaragua,4of7fctqcqbj3rai95vd4jk26,1995-03-30,Puerto La Cruz,1er0616j12s9yghuzvsx76d5v,Venezuela,183.0,82.0,Male,active,NaN
3,2w0vvlgz7ib4f2dvkinycj55h,Leandro,Leite Mateus,Leandro Leite,player,Midfielder,2018-11-24T20:01:10Z,Brazil,2vufyvpoxd9lfl9f6vpp7tz6y,1982-12-27,Piracanjuba,2vufyvpoxd9lfl9f6vpp7tz6y,Brazil,181.0,75.0,Male,active,right
4,eyas8ovmtmcmkilzd9lox4hxx,Sergey,Balanovich,S. Balanovich,player,Midfielder,2018-12-02T15:31:47Z,Belarus,2vn7xzv9g752y1h5ufjzr27i8,1987-08-29,Pinsk,2vn7xzv9g752y1h5ufjzr27i8,Belarus,176.0,70.0,Male,active,right


# Clean person.csv

In [5]:
person = person.loc[person['type'] == 'player']
print('The number of remaining rows is {}'.format(person.shape[0]))
person = person.drop_duplicates()
print('The duplicates are dropped, the shape afterward is {}.'.format(person.shape))

The number of remaining rows is 44201
The duplicates are dropped, the shape afterward is (44200, 18).


In [6]:
person['dateOfBirth'] = pd.to_datetime(person['dateOfBirth'])
person['lastUpdated'] = pd.to_datetime(person['lastUpdated'])
# person.head()

In [7]:
# add a column about the ages.
person['age'] = 2020 - person['dateOfBirth'].dt.year

# add a column about the year of late update.
person['lastUpdated_year'] = person['lastUpdated'].dt.year

# distplot of ages
# chart = sns.distplot(person['age'])
# chart.set_title('Ages Distribution')

# person.describe()

# Clean membership.csv

In [8]:
membership = membership.drop_duplicates()
membership = membership.loc[membership['role']=='player']
print('The shape afterward is {}.'.format(membership.shape))

The shape afterward is (250305, 10).


In [9]:
membership.loc[membership['active']=='yes', 'endDate'] = '2019-1-1'
membership = membership.dropna(subset = ['endDate']) # drop nan data
print('There are {} rows left'.format(membership.shape[0]))
# membership.head()

There are 247915 rows left


In [10]:
#membership.loc[membership['endDate']=='201-05-04']
membership.loc[membership['startDate']=='0201-05-04', 'startDate'] = '2010-05-04'
membership.loc[membership['startDate']=='0011-01-01', 'startDate'] = '2011-01-01'

membership.loc[membership['endDate']=='0010-12-30', 'endDate'] = '2010-12-30'
membership.loc[membership['endDate']=='0201-04-15', 'endDate'] = '2010-04-15'
membership.loc[membership['endDate']=='0204-08-01', 'endDate'] = '2004-08-01'

membership['startDate'] = pd.to_datetime(membership['startDate'])
membership['endDate'] = pd.to_datetime(membership['endDate'])
membership.head()

,id,contestantId,contestantType,contestantName,active,startDate,endDate,role,type1,transferType
0,100cbzhz9d4n7p6lzkejh9rl5,59viwky8c7g3bq3japwgwn4z4,club,FC Rapperswil-Jona,yes,2018-07-17,2019-01-01,player,men,NaN
1,100cbzhz9d4n7p6lzkejh9rl5,6kxut8wiq1w4hr89ipl3bx63h,national,Bosnia-Herzegovina Under 21,no,2015-01-01,2015-12-31,player,youth,NaN
2,100gtoc2x0ica9p01rv3srmxh,cfuo584og02t9erol4nqofzrz,club,Black Leopards FC,yes,2014-01-01,2019-01-01,player,men,Transfer
3,100jsd7342gty8k0d6tte4hqt,81lpho34qr3dad0xzalq0abja,club,FK Crvena Zvezda Beograd,no,2010-01-01,2013-06-30,player,men,Transfer
4,100jsd7342gty8k0d6tte4hqt,dr4emys1xpm2h54oid0q5y2nx,club,OFI FC,no,2016-08-03,2017-01-15,player,men,Transfer


# Clean stat.csv

In [11]:
stat = stat.drop_duplicates()
print('The shape afterward is {}.'.format(stat.shape))

The shape afterward is (750230, 19).


In [12]:
stat.loc[stat['penaltyGoals'] == 58, 'penaltyGoals'] = 0

In [13]:
stat.loc[stat['minutesPlayed'] < 0, 'minutesPlayed'] =  abs(stat['minutesPlayed'])

In [48]:
# membership.loc[(membership['contestantName']=='FC Barcelona')]
# 5cfw7nza8d1dfzatr3noh5amc

# Select Data

In [14]:
#!/user/local/bin/python2.7
# -*- coding:utf-8 -*-
import os
def mkdir(path):
    # remove space
    path=path.replace(' ','')
    # remove \\
    path=path.rstrip("\\")
    # check if path exists
    isExists=os.path.exists(path)
    
    if not isExists:
        # if does not exist, create the folder
        '''
        os.mkdir(path)与os.makedirs(path)的区别是,当父目录不存在的时候os.mkdir(path)不会创建，os.makedirs(path)则会创建父目录
        '''
        #coding as utf-8
        os.makedirs(path) 
        print (path+'is successfully created')
        return True
    else:
        # if exists, create nothing
        print (path+' already exists')
        return False

In [15]:
def team_select(team, season):
    year = int(season[0:4])
    
    # select membership
    membership_team = membership.loc[(membership['contestantName'] == team)&((membership['startDate'].dt.year <= year)&(membership['endDate'].dt.year > year))]
    
    # select players
    person_team = person.loc[person['id'].isin(membership_team['id'])]
    
    # adjust player ages for the selected season
    person_team['age'] = person_team['age'] - (2020 - int(season[5:9]))
    
    # select stats
    stat_team = stat.loc[(stat['id'].isin(membership_team['id'])) & ((stat['tournamentCalendarName'] == season)|(stat['tournamentCalendarName'] == season[5:9]))&(stat['isFriendly'] == 'no')]
    
    # set minutes played for team members
    person_team['minutesPlayed'] = person_team.id.apply(lambda x: np.sum(stat_team.loc[stat_team['id'] == x]['minutesPlayed']))
    
    # select 16 players (at most)
    time_sort = np.sort(person_team['minutesPlayed'])
    time_sort = time_sort[::-1]
    if time_sort.shape[0] >= 16:
        team_size = 16
    else:
        team_size = time_sort.shape[0]

    person_team = person_team.loc[person_team['minutesPlayed'] >= time_sort[team_size-1]]
    
    # sort players by nationality
    group_nation = person_team.groupby('nationality')
    p = pd.DataFrame()
    for group_name, group_df in group_nation:
        p = pd.concat([p,group_df])
    person_team = p
    
    # update membership and stat
    membership_team = membership.loc[membership['id'].isin(person_team['id'])]
    stat_team = stat_team.loc[stat_team['id'].isin(person_team['id'])]
    
    # select club and national memberships
    membership_club = membership_team.loc[membership_team['contestantType'] == 'club']
    membership_national = membership_team.loc[membership_team['contestantType'] == 'national']
    
    return membership_club, membership_national, person_team, stat_team

In [80]:
def career_search(person, membership, season):
    # define 10 seasons
    seasons = seasons_10(season)
    # print('The seasons are from {} to {}'.format(seasons[0], seasons[9]))
    
    # filter membership
    start = int(seasons[0][0:4])
    end = int(seasons[9][5:9])
    membership = membership.loc[((membership['startDate'].dt.year >= start)&(membership['startDate'].dt.year <= end))|((membership['endDate'].dt.year >= start)&(membership['endDate'].dt.year <= end))]
    # print(membership['contestantName'].unique())
    
    # search memberships by season
    col = np.append(['matchName'],seasons)
    career_paths = pd.DataFrame(columns = col) 
    team_names = []
    for i in person['id'].values:
        career = {'matchName': person.loc[person['id'] == i]['matchName'].values[0]}
        for s in seasons:
            year = int(s[0:4])
            # search for membership of the season
            m = membership.loc[(membership['id'] == i) & ((membership['startDate'].dt.year <= year) & (membership['endDate'].dt.year > year))]
            # print(m)
            m_name = m['contestantName'].values  # get team name
            
            # print(m_name)
            if m_name.shape[0]>1:
                m_name = m_name[0:1]
                
            if m_name.shape[0]>=1:
                # transfer Youth into Men
                if 'Under' in str(m_name[0]):
                    m_name[0] = str(m_name[0]).split(' Under ')[0]
                if 'II' in str(m_name[0]):
                    m_name[0] = str(m_name[0]).split(' II')[0]
                team_names.append(m_name[0])  # append team name into list
                
            if len(m_name) == 0:
                career[s] = '/'
            else:
                career[s] = str(m_name[0])
        career_paths = career_paths.append(career,ignore_index=True)
        # print(career)
        
    # transfer team_names into Series    
    team_names = pd.Series(pd.Series(team_names).unique())
    # print(team_names)
    return seasons, career_paths, team_names

In [77]:
# str = 'Liverpool FC Under 21'
# if 'Under' in str:
#     print(str.split(' Under ')[0])
# 
# str = 'Liverpool FC II'
# print(str.split(' II')[0])

In [70]:
# search for diversity
def diversity (person, club_names, national_names):
    age_div = person['age'].unique().shape[0]
    club_div = club_names.shape[0]
    national_div = national_names.shape[0]
    return age_div, club_div, national_div

In [71]:
# search for cohesion
def cohesion(team, seasons, paths_club, paths_national):
    seasons_re = seasons[::-1]
    
    # current cohesion
    current_co = []
    
    # see how many seasons current players have played for the team
    for index,row in paths_club.iterrows():
        current = 0
        for s in seasons_re:
            if row[s] == team:
                current = current+1
            else:
                break
        current_co = np.append(current_co,current)
    current_co = np.sort(current_co)
    
    # current cohesion years among different group sizes
    current_16 = current_co[0]
    current_14 = current_co[2]
    current_12 = current_co[4]
    current_10 = current_co[6]
    current_8 = current_co[8]
    current_co = {'curr_16':current_16, 'curr_14':current_14, 'curr_12':current_12, 'curr_10':current_10, 'curr_8':current_8}
    # print (current_co)
    
    # past cohesion of clubs
    past_co = 0
    # for every season, check if there's overlap besides selected team
    for s in seasons:
        club_unique = paths_club[s].value_counts()
        for i, v in club_unique.iteritems():
            if v>1:
                if (i != '/') & (i != team):
                    past_co = past_co+1
            else:
                break
    # print(past_co)
    
    # cohesion of national teams
    national_co = 0
    # for every season, check if there's national team overlap
    for s in seasons:
        national_unique = paths_national[s].value_counts()
        for i, v in national_unique.iteritems():
            if v>1:
                if i != '/':
                    national_co = national_co+1
            else:
                break
    
    return current_co, past_co, national_co

In [72]:
# define 10 seasons
def seasons_10(season):
    seasons = [season]
    for i in range(1,10):
        year = int(season[5:9])-i
        s = str(year-1) + '/' + str(year)
        seasons.append(s)
    seasons = seasons[::-1]
    return seasons

In [73]:
# see diversity and cohesion
def data_select(team, season):
    # print('Selecting data about {} in season {} ...'.format(team, season))
    
    # select memberships and players
    membership_club, membership_national, person, stat_team = team_select(team, season)
    
    # select career paths for both club and national team
    seasons, paths_club, club_names = career_search(person, membership_club, season)
    seasons, paths_national, national_names = career_search(person, membership_national, season)
    
    # calculate team diversity
    age_div, club_div, national_div = diversity(person, club_names, national_names)
    # print(age_div, club_div, national_div)
    
    #calculate team cohesion
    current_co, past_co, national_co = cohesion(team, seasons, paths_club, paths_national)
    # print(current_co, past_co, national_co)
    
    return age_div, club_div, national_div, current_co, past_co, national_co

In [74]:
def normalization(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range

In [83]:
seasons = seasons_10('2018/2019')
teams = ['Arsenal FC', 'Tottenham Hotspur FC', 'Chelsea FC', 'Newcastle United FC']
np.set_printoptions(precision=3,suppress=True)
# input ranks
arsenal = [3, 4, 3, 4, 4, 3, 2, 4, 6, 5]
tottenham = [4, 5, 4, 5, 6, 5, 3, 2, 3, 4]
chelsea = [1, 2, 6, 3, 3, 1, 10, 1, 5, 3]
newcastle = [21, 12, 5, 16, 10, 15, 18, 21, 10, 13]

arsenal_norm = normalization(arsenal)
tottenham_norm = normalization(tottenham)
chelsea_norm =normalization(chelsea)
newcastle_norm = normalization(newcastle)

print('Normalized Arsenal Rank: {}'.format(arsenal_norm))
print('Normalized Tottenham Rank: {}'.format(tottenham_norm))
print('Normalized Chelsea Rank: {}'.format(chelsea_norm))
print('Normalized Newcastle Rank: {}'.format(newcastle_norm))
# put data into dictionaries
i = 0
rank_arsenal= {}
rank_tottenham = {}
rank_chelsea = {}
rank_newcastle = {}
for season in seasons:
    rank_arsenal[season] = arsenal[i]
    rank_tottenham[season] = tottenham[i]
    rank_chelsea[season] = chelsea[i]
    rank_newcastle[season] = newcastle[i]
    i+=1

# sort target data into dictionaries
for team in teams:
    print('Proccessing {}...\n\n'.format(team))
    data = {'age_div':[],'club_div':[],'national_div':[],
        'curr_16':[],'curr_14':[],'curr_12':[],'curr_10':[],'curr_8':[],
        'past_co':[],'national_co':[]}
    data_norm = {}
    for season in seasons:
        age_div, club_div, national_div, current_co, past_co, national_co = data_select(team,season)
        data['age_div'].append(age_div)
        data['club_div'].append(club_div)
        data['national_div'].append(national_div)
        
        data['curr_16'].append(current_co['curr_16'])
        data['curr_14'].append(current_co['curr_14'])
        data['curr_12'].append(current_co['curr_12'])
        data['curr_10'].append(current_co['curr_10'])
        data['curr_8'].append(current_co['curr_8'])
        
        data['past_co'].append(past_co)
        data['national_co'].append(national_co)
    for key in data:
        data_norm[key] = normalization(data[key])
    print(data)
    print('\n\n')
    print(data_norm)
    print('\n\n\n')

Normalized Arsenal Rank: [0.25 0.5  0.25 0.5  0.5  0.25 0.   0.5  1.   0.75]
Normalized Tottenham Rank: [0.5  0.75 0.5  0.75 1.   0.75 0.25 0.   0.25 0.5 ]
Normalized Chelsea Rank: [0.    0.111 0.556 0.222 0.222 0.    1.    0.    0.444 0.222]
Normalized Newcastle Rank: [1.    0.438 0.    0.688 0.312 0.625 0.812 1.    0.312 0.5  ]
Proccessing Arsenal FC...




/Users/zzy/opt/anaconda3/envs/py3iaml/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/zzy/opt/anaconda3/envs/py3iaml/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/zzy/opt/anaconda3/envs/py3iaml/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing 

{'age_div': [9, 8, 9, 9, 9, 10, 12, 10, 11, 10], 'club_div': [15, 17, 25, 26, 25, 32, 30, 21, 24, 26], 'national_div': [10, 10, 12, 11, 8, 9, 10, 7, 10, 12], 'curr_16': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'curr_14': [1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0], 'curr_12': [2.0, 3.0, 1.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0], 'curr_10': [2.0, 3.0, 2.0, 2.0, 3.0, 1.0, 2.0, 3.0, 2.0, 1.0], 'curr_8': [3.0, 4.0, 3.0, 3.0, 4.0, 2.0, 3.0, 3.0, 4.0, 2.0], 'past_co': [0, 0, 0, 1, 2, 2, 2, 0, 0, 6], 'national_co': [3, 7, 4, 6, 23, 16, 17, 13, 25, 6]}



{'age_div': array([0.25, 0.  , 0.25, 0.25, 0.25, 0.5 , 1.  , 0.5 , 0.75, 0.5 ]), 'club_div': array([0.   , 0.118, 0.588, 0.647, 0.588, 1.   , 0.882, 0.353, 0.529,
       0.647]), 'national_div': array([0.6, 0.6, 1. , 0.8, 0.2, 0.4, 0.6, 0. , 0.6, 1. ]), 'curr_16': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'curr_14': array([0., 1., 0., 0., 1., 0., 1., 0., 0., 0.]), 'curr_12': array([0.5, 1. , 0. , 0.5, 0.5, 0. 